In [1]:
from src.utils.extract_utils import gather_activations, gather_activations_from_dataset

/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch, transformers, accelerate, einops

In [3]:
from src.utils.model_utils import load_gpt_model_and_tokeniser

In [4]:
model, tokenizer, MODEL_CONFIG = load_gpt_model_and_tokeniser(model_name="gpt2-xl")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [5]:
resid_td = gather_activations(
    "Good day sir, how are you doing today?",
    MODEL_CONFIG['layer_hook_names'],
    model,
    tokenizer,
)

In [20]:
len(resid_td[MODEL_CONFIG['layer_hook_names'][0]].output)

2

In [28]:
torch.stack([resid_td['transformer.h.0'].output[0], resid_td['transformer.h.0'].output[0]]).shape

torch.Size([2, 1, 10, 1600])

In [23]:
attn_td = gather_activations(
    "Good day sir, how are you doing today?",
    MODEL_CONFIG['attn_hook_names'],
    model,
    tokenizer,
)

In [24]:
attn_td['transformer.h.0.attn.c_proj'].output[0].shape

torch.Size([10, 1600])

In [22]:
#desired shape: (batch_size, layer, n_tokens, dim)
tensor = torch.stack([resid_td[layer].output for layer in MODEL_CONFIG['attn_hook_names']], dim=1)

KeyError: 'transformer.h.0.attn.c_proj'

In [ ]:
einops.rearrange()

torch.Size([1, 48, 10, 1600])

In [8]:
td[MODEL_CONFIG['attn_hook_names'][1]].output[0].shape

torch.Size([10, 1600])

In [9]:
dir(td[MODEL_CONFIG['attn_hook_names'][0]])

['__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'close',
 'input',
 'layer',
 'registered_hook',
 'stop']

In [6]:
activations = gather_activations_from_dataset(
    ["Good day sir, how are you doing today?", "I am doing well, thank you for asking."],
    ["attn_hook_names"],
    model,
    tokenizer,
    MODEL_CONFIG,
    2,
    False,
    True
)

torch.Size([48, 10, 1600])
torch.Size([48, 10, 1600])


In [8]:
activations['attn_hook_names'][0].shape

torch.Size([48, 1, 1600])

In [7]:
activations

{'attn_hook_names': {0: tensor([[[-1.7877e-02,  3.2479e-02, -1.4434e-02,  ..., -6.7941e-02,
             7.5504e-02,  1.6733e-02]],
  
          [[ 1.3076e-01, -6.2840e-02, -4.5099e-02,  ...,  2.1293e-01,
            -4.2480e-02,  3.4764e-02]],
  
          [[ 2.0405e-02,  4.4779e-02, -3.6921e-02,  ..., -3.0827e-02,
            -1.4238e-01, -3.7328e-02]],
  
          ...,
  
          [[-6.4238e-01, -6.7807e-01, -3.2924e-01,  ..., -1.6132e+01,
            -6.1254e-01, -1.1006e+00]],
  
          [[-1.4672e-01,  9.3154e-02, -1.3612e-01,  ..., -3.6234e+01,
            -3.1323e-01, -4.6978e-01]],
  
          [[ 3.5768e-01,  1.9606e-01,  8.6479e-01,  ..., -1.1085e+01,
            -4.4635e-01,  5.1237e-02]]], device='cuda:0',
         grad_fn=<ReshapeAliasBackward0>),
  1: tensor([[[ 8.4371e-03,  2.3719e-02,  1.0827e-02,  ..., -3.3776e-02,
             4.6411e-02, -2.2447e-02]],
  
          [[-4.3184e-03, -4.8394e-02, -2.3387e-01,  ..., -1.5990e-01,
             1.3697e-02,  1.2034e-01]]